In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raultruco/cryptowatson-indicators#egg=cryptowatson-indicators
from cryptowatsonindicators.fng import Fng
import requests
requests.packages.urllib3.disable_warnings()
import pprint
pprint = pprint.PrettyPrinter(indent=2, sort_dicts=False, compact=False).pprint   # pprint with defaults

### Global variables

In [ ]:
ticker_symbol = "BTCUSDT"    # currently only works with BTCUSDT
start_date = None            # Limt time series to start at specific date or None to use all the history
binance_api_key = ''         # use your binance keys or leave blank to use Binance's testnet
binance_secret_key = ''
get_current_index = True
get_index_at = True
plot_fng_and_ticker_chart = True
run_simulator = True

fng = Fng(start_date=start_date, ticker_symbol=ticker_symbol, binance_api_key=binance_api_key, binance_secret_key=binance_secret_key)

### Get Fear and Greed index

In [ ]:
if get_current_index:
    fng_index = fng.get_current_fng()
    print('Current FnG Index:')
    pprint(fng_index)

In [ ]:
if get_index_at:
    at_date = '01/04/2022'    # date when look up the band
    price_at_date = 30000     # Price of BTC at date
    fng_index = fng.get_fng(at_date=at_date)
    print(f"FnG Index at {at_date}:")
    pprint(fng_index)

### Plot Fear and Greed History

In [ ]:
if plot_fng_and_ticker_chart:
    fng.plot_fng_and_ticker_price()

### Simulator

Usage...

In [ ]:

start_date = '01/01/2020'    # start date of the simulation
buy_frequency = 7            # Number of days between purchases
buy_amount = 100             # Amount purchased in standard DCA

if run_simulator:
    pass
